In [2]:
import pandas as pd
import numpy as np

In [3]:
pesos = pd.read_excel("pesos.xlsx", header = 0)
jugadores = pd.read_excel("CA.xlsx", header = 0)
pesos.set_index("Atributo", inplace = True)

In [3]:
pie = {"Very Weak":2,"Weak":6,"Reasonable":10,"Strong":16,"Fairly Strong":18,"Very Strong":20}
pos_dict = {'GK':'GK','D (R)':'DRL', 'D (L)':'DRL', 'D (C)':'DC', 'WB (R)':'WBRL', 'WB (L)':'WBRL', 'DM':'DM', 'M (L)':'MRL', 'M (R)':'MRL','M (C)':'MC', 'AM (L)':'AMRL', 'AM (R)':'AMRL','AM (C)':'AMC','ST (C)':'SC'}
CA = {}
atributos = ['Cor', 'Cro', 'Dri', 'Fin', 'Fir', 'Fre', 'Hea', 'Lon', 'L Th',
       'Mar', 'Pas', 'Pen', 'Tck', 'Tec', 'Agg', 'Ant', 'Bra', 'Cmp', 'Cnt',
       'Dec', 'Det', 'Fla', 'Ldr', 'OtB', 'Pos', 'Tea', 'Vis', 'Wor', 'Acc',
       'Agi', 'Bal', 'Jum', 'Nat', 'Pac', 'Sta', 'Str', 'Aer', 'Cmd', 'Com',
       'Ecc', 'Han', 'Kic', '1v1', 'Pun', 'Ref', 'TRO', 'Thr','WeakFoot']

In [4]:
def obtener_pos(POS):
    lista_comas = POS.split(",")
    lista_final = []
    for i in lista_comas:
            if len(i.split("/")) > 1:
                lista_varios = i.strip().split(" ")
                lista_pos = lista_varios[0].split("/")
                for i in lista_pos:
                    final = lista_varios[-1]
                    if len(final[final.rfind('(')+1:final.rfind(')')]) > 1:
                        for letra in final[final.rfind('(')+1:final.rfind(')')]:
                            lista_final.append(i + ' ' + ('('+letra+')'))
                    else:
                        lista_final.append(i + ' ' + (final))
            else:
                lista_varios = i.strip().split(" ")
                final = lista_varios[-1]
                if len(final[final.rfind('(')+1:final.rfind(')')]) > 1:
                        for letra in final[final.rfind('(')+1:final.rfind(')')]:
                            lista_final.append(lista_varios[0] + ' ' + ('('+letra+')'))
                else:
                    lista_final.append(i)
    return [i.strip() for i in lista_final]

In [5]:
jugadores['WeakFoot'] = [min(pie[jugadores.iloc[i]["Left Foot"]],pie[jugadores.iloc[i]["Right Foot"]]) for i in range(len(jugadores))]

In [6]:
for i in range(len(jugadores)):
    denominador = []
    posibles_scores = []
    for pos in obtener_pos(jugadores.iloc[i]["Position"]):
        denominador = []
        for atr in atributos:
            pos_final = pos_dict[pos]
            if type(pesos[pos_final][atr]) == type(pesos["GK"]["Pas"]):
                denominador.append(jugadores.iloc[i][atr]*max(pesos[pos_final][atr]))
            else:
                denominador.append(jugadores.iloc[i][atr]*pesos[pos_final][atr])
        CA[jugadores.iloc[i].Name + " " + '[' + pos + ']'] = round((sum(denominador)/pesos[pos_final].sum())*18.139 - 97.871)

In [7]:
CurrentAbility = pd.DataFrame(columns = ["Nombre","Posición","CA"])
CurrentAbility["Nombre"] = [i[0:i.rfind('[')-1].strip() for i in CA.keys()]
CurrentAbility["Posición"] = [i[i.rfind('[')+1:i.rfind(']')].strip() for i in CA.keys()]
CurrentAbility["CA"] = list(CA.values())

In [8]:
df_atr = pd.read_excel("Atributos.xlsx", header = 0)

In [9]:
df_plantilla = CurrentAbility.merge(df_atr, left_on = "Nombre", right_on = "Name")
df_plantilla = df_plantilla[[i for i in df_plantilla.columns if i not in ["Nombre","Position"]]]

In [10]:
df_plantilla.to_excel("PLANTILLA.xlsx")

In [5]:
#df1 = pd.read_excel("BBDD1.xlsx", header = 0) #ESTO ESTÄ MAL
df_final = pd.read_excel("BBDD2.xlsx", header = 0, na_values = '-')

In [6]:
def cambioApps(df_final):
    valores = []
    for i in range(len(df_final)):
        if df_final.Apps.astype('str')[i].rfind(' ') != -1:
            valores.append(float(df_final.Apps.astype('str')[i][:df_final.Apps.astype('str')[i].rfind(' ')]))
        else:
            valores.append(float(df_final.Apps.astype('str')[i]))
    return valores

In [7]:
df_final.fillna(0, inplace = True)
df_final['Apps'] = cambioApps(df_final)
df_final = df_final[df_final.Apps > 10]
df_final = df_final[['Name','Apps','Mins', 'Mins/Gm', 'Position','Height', 'Weight', 'Age','Transfer Value','Av Rat','Gls','Gls/90','Shot %','Pen/R','xG','Ch C/90','Asts/90','K Ps/90','Pas %','Cr C/A','Drb/90','Distance','Hdr %','K Tck','Fls','Int/90','Clear','Con/90','xSv %','Sv %','Svh', 'Svt', 'Svp','Pens Saved Ratio','PoM','Aer A/90','Off','Based','Tck R']]
df_final.columns = ['Name','Apps','Mins', 'Mins/Gm', 'Position','Height', 'Weight', 'Age','Transfer Value','Av Rat','Gls','Gls/90','Shot %','Pen/R','xG','Ch C/90','Asts/90','K Ps/90','Pas %','Cr C/A','Drb/90','Distance','Hdr %','K Tck','Fls','Int/90','Clear','Con/90','xSv %','Sv %','Svh', 'Svt', 'Svp','Pens Saved Ratio','PoM','Aer A/90','Off','Based', 'Tck R']
#df_final = df_final[df_final['Transfer Value'] != 'Not for Sale']
df_final.reset_index(inplace = True)
df_final.drop(['Transfer Value','index'], axis = 1, inplace = True)

In [8]:
columnas = {'Name':[],'POS':[]}
for i in range(len(df_final)):
    for pos in obtener_pos(df_final.iloc[i]["Position"]):
        columnas['Name'].append(df_final.iloc[i]["Name"])
        columnas['POS'].append(pos)
posiciones = pd.DataFrame(data = columnas)

In [9]:
variables = df_final.merge(posiciones, on = ["Name"])
variables.drop("Position", axis = 1, inplace = True)
variables['Height'] = [float(variables.iloc[i].Height[0:variables.iloc[i].Height.rfind(' ')]) for i in range(len(variables))]
variables['Weight'] = [float(variables.iloc[i].Weight[0:variables.iloc[i].Weight.rfind(' ')]) for i in range(len(variables))]
variables['Distance'] = [float(variables.iloc[i].Distance[0:variables.iloc[i].Distance.rfind('k')]) for i in range(len(variables))]

In [10]:
variables['Saves'] = variables['Svh'] + variables['Svt'] + variables['Svp']
variables['Mins/Gm'] = variables['Mins']/variables['Apps']
variables['Saves/xSv%'] = variables['Saves']/variables['xSv %']
variables['Gls/xG'] = variables['Gls']/variables['xG']
variables['Dist/Mins'] = variables['Distance']/variables['Mins']
variables.replace([np.inf, -np.inf], np.nan,inplace=True)
variables.fillna(0,inplace = True)

In [11]:
porteros = variables[variables.POS == 'GK'][["Name", "POS", "Based", "Apps", "Mins", "Mins/Gm", "Height", "Weight", "Age", "Av Rat", "Con/90", "Svh", "Svt", "Saves", "Svp", "Sv %", "xSv %", "Saves/xSv%", "Pens Saved Ratio", "PoM"]]
defensas = variables[(variables.POS == 'D (C)') | (variables.POS == 'D (L)') | (variables.POS == 'D (R)') | (variables.POS == 'WB (L)') | (variables.POS == 'WB (R)')][["Name", "POS", "Based", "Apps", "Mins", "Mins/Gm", "Height", "Weight", "Age", "Av Rat", "Distance", "Dist/Mins", "Hdr %", "Aer A/90", "K Tck", "Tck R", "Fls", "Int/90", "Pas %", "Clear", "PoM"]]
medios = variables[(variables.POS == 'M (C)') | (variables.POS == 'M (L)') | (variables.POS == 'M (R)') | (variables.POS == 'DM')][["Name", "POS", "Based", "Apps", "Mins", "Mins/Gm", "Height", "Weight", "Age", "Av Rat", "Gls", "Gls/90", "xG", "Gls/xG", "Ch C/90", "Asts/90", "K Ps/90", "Pas %", "Cr C/A", "Distance", "Dist/Mins", "Hdr %", "Aer A/90", "K Tck","Tck R", "Fls", "Int/90", "Clear", "PoM"]]
delanteros = variables[(variables.POS == 'AM (C)') | (variables.POS == 'AM (R)') | (variables.POS == 'AM (L)') | (variables.POS == 'ST (C)')][["Name", "POS", "Based", "Apps", "Mins", "Mins/Gm", "Height", "Weight", "Age", "Av Rat", "Gls", "Gls/90", "xG", "Gls/xG", "Shot %", "Ch C/90", "Asts/90", "K Ps/90", "Drb/90", "Distance", "Dist/Mins", "Hdr %", "Aer A/90", "Pen/R", "Off", "PoM"]]

In [12]:
#SACAMOS EXCELS
porteros.to_excel("porteros_cambio.xlsx")
defensas.to_excel("defensas_cambio.xlsx")
medios.to_excel("medios_cambio.xlsx")
delanteros.to_excel("delanteros_cambio.xlsx")

In [65]:
#RED NEURONAL
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [95]:
#Porteros
dep = porteros[['Value']]
indep = porteros[['Height', 'Weight', 'Age', 'Av Rat', 'Con/90',
       'xSv %', 'Pens Saved Ratio', 'PoM','Saves', 'Saves/xSv%']]
#Separamos train/test
X_train,X_test, y_train, y_test = train_test_split(indep, dep, test_size = 0.3, random_state = 142)
model = LinearRegression()
model.fit(X_train, y_train)
cv_score = cross_val_score(model, X_test, y_test, cv = 10)
np.mean(cv_score)

-167.07875935984117

In [92]:
len(porteros)

479